In [ ]:
########## Dependencies
##### define input directory
img_dir = "/mnt/local/data2/Bootsma/2D_CTC/src/analysis/publication_code/test_data/"

##### Libraries
import os
import sys
import nd2
import numpy as np
import pandas as pd

sys.path.append('../src/') 
import SEE_TC as ctc


In [ ]:
########## Define input
files = ctc.parse_nd2_paths(img_dir, "nd2", recursive=True)
files = [s.replace(".nd2", "") for s in files]
print(len(files))


In [ ]:
########## Process images
tile_stack_raw = [] # initialize output object

for i in range(len(files)):
    print("-------------"+str(i)+"------------")
    file_i = files[i]
    img_path_i = file_i+".nd2"

    if os.path.isfile(file_i+".image_metadata.csv"):
        meta_df = pd.read_csv(file_i+".image_metadata.csv")
        if 'zoom_optics' in meta_df.columns and 'date_nd2' in meta_df.columns:
            print("skipping sample ("+file_i+"), already processed")
            print("###############\n")            
            continue
    if os.path.isfile(img_path_i) == False:
        print("skipping sample ("+file_i+") no input")
        print("###############\n")            
        continue
    print(file_i)
    img_i, CHANNELS = ctc.extract_all_array_nd2(img_path_i)

    img_i = np.transpose(img_i, (1,2,0)) # channel last
    print("Image shape (raw): "+str(img_i.shape))
    df = pd.DataFrame(CHANNELS, columns=['channel_ID'])
    df['sample_ID'] = file_i
    df['dim_Y'] = img_i.shape[0]
    df['dim_X'] = img_i.shape[1]

    # extract, parse, and merge exposure settings
    with nd2.ND2File(img_path_i) as ndfile:
        text_unparsed = ndfile.text_info
        cap_settings = text_unparsed['capturing']
        c_names = ndfile._channel_names
        text_dict, exposure_df = ctc.parse_exposure_times(cap_settings, c_names)

    merged_df = pd.merge(exposure_df, df, on='channel_ID', how='inner')
    if merged_df.shape[0] == 0:
        merged_df = df.copy()
    merged_df['device'] = text_dict['Device']
    merged_df['serial_number'] = text_dict['SerialNumber']
    merged_df['zoom_optics'] = text_unparsed['optics']
    merged_df['date_nd2'] = text_unparsed['date']
    # Output result
    print(merged_df)

    merged_df.to_csv(file_i+".image_metadata.csv", index=False)

